## AUZRE Custom Vision 설치

In [1]:
!pip install azure-cognitiveservices-vision-customvision

  Using cached azure_cognitiveservices_vision_customvision-3.1.1-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached azure_mgmt_core-1.5.0-py3-none-any.whl.metadata (4.3 kB)
Using cached azure_cognitiveservices_vision_customvision-3.1.1-py2.py3-none-any.whl (63 kB)
Using cached azure_mgmt_core-1.5.0-py3-none-any.whl (30 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Azure CV 키 인증

In [18]:
# CV 엔드포인트, 키, CV-prediction 키, resource(CV 속성탭에 위치함)
import os
from dotenv import load_dotenv
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

load_dotenv()
ENDPOINT = os.getenv("VISION_TRAINING_ENDPOINT")
training_key = os.getenv("VISION_TRAINING_KEY")
prediction_key = os.getenv("VISION_PREDICTION_KEY")
prediction_endpoint = os.getenv("VISION_PREDICTION_ENDPOINT")
prediction_resource_id = os.getenv("VISION_PREDICTION_RESOURCE_ID")

In [19]:
# 클라이언트 인증
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(prediction_endpoint, prediction_credentials)

## 프로젝트 생성

In [20]:
domains = trainer.get_domains()
obj_detection_domain = next(d for d in domains if d.type == "ObjectDetection" and "General" in d.name)

# 🔹 프로젝트 생성
publish_iteration_name = "6a060-fork-scissors-Detection"

project = trainer.create_project(
    name=publish_iteration_name,
    domain_id=obj_detection_domain.id,
    classification_type="Multiclass",  # 또는 "Multilabel"
    target_resource_id=prediction_resource_id,
    description="Custom Vision 포크-가위 객체감지"
)

print(f"프로젝트 생성 완료: {project.name} (ID: {project.id})")

프로젝트 생성 완료: 6a060-fork-scissors-Detection (ID: 2f63c6ab-b3ab-4cee-8123-77de715b4d28)


## 사진 태그생성

In [21]:
# Make two tags in the new project
fork_tag = trainer.create_tag(project.id, "fork")
scissors_tag = trainer.create_tag(project.id, "scissors")

In [23]:
fork_image_regions = {
    "fork_1": [ 0.145833328, 0.3509314, 0.5894608, 0.238562092 ],
    "fork_2": [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "fork_3": [ 0.09191177, 0.0682516545, 0.757352948, 0.6143791 ],
    "fork_4": [ 0.254901975, 0.185898721, 0.5232843, 0.594771266 ],
    "fork_5": [ 0.2365196, 0.128709182, 0.5845588, 0.71405226 ],
    "fork_6": [ 0.115196079, 0.133611143, 0.676470637, 0.6993464 ],
    "fork_7": [ 0.164215669, 0.31008172, 0.767156839, 0.410130739 ],
    "fork_8": [ 0.118872553, 0.318251669, 0.817401946, 0.225490168 ],
    "fork_9": [ 0.18259804, 0.2136765, 0.6335784, 0.643790841 ],
    "fork_10": [ 0.05269608, 0.282303959, 0.8088235, 0.452614367 ],
    "fork_11": [ 0.05759804, 0.0894935, 0.9007353, 0.3251634 ],
    "fork_12": [ 0.3345588, 0.07315363, 0.375, 0.9150327 ],
    "fork_13": [ 0.269607842, 0.194068655, 0.4093137, 0.6732026 ],
    "fork_14": [ 0.143382356, 0.218578458, 0.7977941, 0.295751631 ],
    "fork_15": [ 0.19240196, 0.0633497, 0.5710784, 0.8398692 ],
    "fork_16": [ 0.140931368, 0.480016381, 0.6838235, 0.240196079 ],
    "fork_17": [ 0.305147052, 0.2512582, 0.4791667, 0.5408496 ],
    "fork_18": [ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
    "fork_19": [ 0.219362751, 0.141781077, 0.5919118, 0.6683006 ],
    "fork_20": [ 0.180147052, 0.239820287, 0.6887255, 0.235294119 ]
}

scissors_image_regions = {
    "scissors_1": [ 0.4007353, 0.194068655, 0.259803921, 0.6617647 ],
    "scissors_2": [ 0.426470578, 0.185898721, 0.172794119, 0.5539216 ],
    "scissors_3": [ 0.289215684, 0.259428144, 0.403186262, 0.421568632 ],
    "scissors_4": [ 0.343137264, 0.105833367, 0.332107842, 0.8055556 ],
    "scissors_5": [ 0.3125, 0.09766343, 0.435049027, 0.71405226 ],
    "scissors_6": [ 0.379901975, 0.24308826, 0.32107842, 0.5718954 ],
    "scissors_7": [ 0.341911763, 0.20714055, 0.3137255, 0.6356209 ],
    "scissors_8": [ 0.231617644, 0.08459154, 0.504901946, 0.8480392 ],
    "scissors_9": [ 0.170343131, 0.332957536, 0.767156839, 0.403594762 ],
    "scissors_10": [ 0.204656869, 0.120539248, 0.5245098, 0.743464053 ],
    "scissors_11": [ 0.05514706, 0.159754932, 0.799019635, 0.730392158 ],
    "scissors_12": [ 0.265931368, 0.169558853, 0.5061275, 0.606209159 ],
    "scissors_13": [ 0.241421565, 0.184264734, 0.448529422, 0.6830065 ],
    "scissors_14": [ 0.05759804, 0.05027781, 0.75, 0.882352948 ],
    "scissors_15": [ 0.191176474, 0.169558853, 0.6936275, 0.6748366 ],
    "scissors_16": [ 0.1004902, 0.279036, 0.6911765, 0.477124184 ],
    "scissors_17": [ 0.2720588, 0.131977156, 0.4987745, 0.6911765 ],
    "scissors_18": [ 0.180147052, 0.112369314, 0.6262255, 0.6666667 ],
    "scissors_19": [ 0.333333343, 0.0274019931, 0.443627447, 0.852941155 ],
    "scissors_20": [ 0.158088237, 0.04047389, 0.6691176, 0.843137264 ]
}

## 이미지 업로드

In [24]:
base_image_location = os.path.join(os.getcwd(), "data/CustomVision-ObjectDetection")
# Go through the data table above and create the images
print ("Adding images...")
tagged_images_with_regions = []

for file_name in fork_image_regions.keys():
    x,y,w,h = fork_image_regions[file_name]
    regions = [ Region(tag_id=fork_tag.id, left=x,top=y,width=w,height=h) ]

    with open(os.path.join (base_image_location, "fork", file_name + ".jpg"), mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

for file_name in scissors_image_regions.keys():
    x,y,w,h = scissors_image_regions[file_name]
    regions = [ Region(tag_id=scissors_tag.id, left=x,top=y,width=w,height=h) ]

    with open(os.path.join (base_image_location, "scissors", file_name + ".jpg"), mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=tagged_images_with_regions))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

Adding images...


## 이미지 업로드 확인

In [25]:
if upload_result.is_batch_successful:
    print("모든 이미지가 성공적으로 업로드 되었습니다.")

모든 이미지가 성공적으로 업로드 되었습니다.


In [26]:
for i in tagged_images_with_regions:
    print(i.name)
    print(i.regions)

fork_1
fork_2
fork_3
fork_4
fork_5
fork_6
fork_7
fork_8
fork_9
fork_10
fork_11
fork_12
fork_13
fork_14
fork_15
fork_16
fork_17
fork_18
fork_19
fork_20
scissors_1
scissors_2
scissors_3
scissors_4
scissors_5
scissors_6
scissors_7
scissors_8
scissors_9
scissors_10
scissors_11
scissors_12
scissors_13
scissors_14
scissors_15
scissors_16
scissors_17
scissors_18
scissors_19
scissors_20


## 프로젝트 학습

In [27]:
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training


In [28]:
iteration.as_dict()

{'id': '1f417731-a060-494b-b2fd-cf2099280706',
 'name': 'Iteration 1',
 'status': 'Completed',
 'created': '2025-03-25T08:43:46.646Z',
 'last_modified': '2025-03-25T08:53:31.781Z',
 'trained_at': '2025-03-25T08:53:31.777Z',
 'project_id': '2f63c6ab-b3ab-4cee-8123-77de715b4d28',
 'exportable': False,
 'domain_id': '9c616dff-2e7d-ea11-af59-1866da359ce6',
 'training_type': 'Regular',
 'reserved_budget_in_hours': 0,
 'training_time_in_minutes': 10}

In [29]:
performance = trainer.get_iteration_performance(project.id, iteration.id)

print(performance.precision, performance.recall, performance.average_precision)

1.0 1.0 1.0


## 현재 반복 게시

In [30]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


## 테스트

In [31]:
notebook_dir = os.getcwd()
base_image_location = "data/CustomVision-ObjectDetection"
image_path = os.path.join(notebook_dir, base_image_location, "test", "test_image.jpg")

In [32]:
# Now there is a trained endpoint that can be used to make a prediction

# Open the sample image and get back the prediction results.
with open(image_path, mode="rb") as test_data:
    results = predictor.detect_image(project.id, publish_iteration_name, test_data)

# Display the results.    
for prediction in results.predictions:
    print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, "
                                       "bbox.top = {2:.2f}, bbox.width = {3:.2f}, "
                                       "bbox.height = {4:.2f}".format(prediction.probability * 100, 
                                                                      prediction.bounding_box.left, 
                                                                      prediction.bounding_box.top, 
                                                                      prediction.bounding_box.width, 
                                                                      prediction.bounding_box.height))

	fork: 99.96% bbox.left = 0.06, bbox.top = 0.21, bbox.width = 0.74, bbox.height = 0.62
	scissors: 1.78% bbox.left = 0.04, bbox.top = 0.13, bbox.width = 0.83, bbox.height = 0.87
